In [13]:
#!pip install OSMPythonTools
#!pip install geopy
#!pip install folium
#!pip install geocoder

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from tqdm import tqdm

In [3]:
df_p = pd.read_excel("해커톤 약 상세정보.xlsx", engine='openpyxl')
df_s = pd.read_excel("해커톤 약 주의사항 부분.xlsx", engine='openpyxl')
df_p['사용상 주의사항'] = df_s['사용상 주의사항 ']

In [4]:
df_p.head()

,Label,약명,용법용량,사용상 주의사항,효능효과,Purpose,성분정보,저장방법
0,0,임팩타민 프리미엄 정,12세 이상 어린이 및 성인: 1일 1회 1~2정 식후 복용,(1) 이 약 및 이 약에 포함된 성분에 과민반응이 있는 환자\n(2) 만 12개월...,"○ 다음경우의 비타민 (E, B1, B2, B6, C)의 보급\n- 육체피로, 임신...","육체피로, 체력저하, 눈의파로, 구내염, 근육통, 관절통, 신경통","니코틴산아미드 50mg, 리보플라빈 50mg, 벤포티아민 50mg, 비오틴 0.25...","기밀용기, 실온보관(1∼30℃)"
1,1,베아제,성인 : 1회 1정 1일 3회 식후에 복용한다.,1. 다음과 같은 사람은 이 약을 복용하지 말 것.\n만 7세 이하의 어린이\n\n...,"소화불량, 식욕감퇴(식욕부진), 과식, 체함, 소화촉진, 소화불량으로 인한 위부팽만감","소화불량, 과식, 체함","시메티콘 40mg, 우르소데옥시콜산 10mg, 판크레아틴장용과립 78.6mg, 판프...",기밀용기
2,2,베아제,성인 : 1회 1정 1일 3회 식후에 복용한다.,1. 다음과 같은 사람은 이 약을 복용하지 말 것.\n만 7세 이하의 어린이\n\n...,"소화불량, 식욕감퇴(식욕부진), 과식, 체함, 소화촉진, 소화불량으로 인한 위부팽만감","소화불량, 과식, 체함","시메티콘 40mg, 우르소데옥시콜산 10mg, 판크레아틴장용과립 78.6mg, 판프...",기밀용기
3,3,씨콜드 코프정,"1일 3회 식후 30분, 1회에 다음 용량을 복용하십시오.\n만 15세 이상 : 1...",1) 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 해열진통제를 복...,"감기의 제증상(여러 증상)(콧물, 코막힘, 재채기, 인후(목구멍)통, 기침, 가래,...","기침, 거래, 인후 (목구멍)통","DL-메틸에페드린염산염 12mg, 구아이페네신 40mg, 슈도에페드린염산염 15mg...","기밀용기, 실온(1～30℃)보관"
4,4,씨콜드 노즈정,"<주간용>\n1일 2회 식후 30분, 1회에 다음 용량을 복용하십시오\n만 15세 ...",(1) 경고\n1) 매일 세잔 이상 정기적으로 술을 마시는 사람이 이 약이나 다른 ...,"감기의 제증상(여러 증상) (콧물, 코막힘, 재채기, 인후(목구멍)통, 오한(춥고 ...","콧물, 코막힘, 재채기","DL-말레인산클로르페니라민 1.25mg, 슈도에페드린염산염 15mg, 아세트아미노펜...","기밀용기, 실온(1～30℃)보관"


# Coordinates Calculation Part

In [5]:
coordinates = pd.read_excel('theshop_gps_info_20210727.xlsx', engine='openpyxl', sheet_name=1)

from geopy.exc import GeocoderTimedOut
from geopy.geocoders import ArcGIS

nom = ArcGIS()

s = nom.geocode(coordinates.iloc[0,1])

In [6]:
coordinates.head(2)

,user_key,pharmercy_name,street_addr1,street_addr2,sido,gugun,dong,dong_code,longitude,latitude
0,1000116885,자이약국,충북 청주시 흥덕구 서현로 32 (가경동),가경자이프라자 120호,충청북도,청주시,가경동,4311374700,127.426605,36.617074
1,1000065193,서현약국,충청북도 청주시 흥덕구 서현북로 64 (가경동),1987번지 서현약국,충청북도,청주시,가경동,4311374700,127.429525,36.621843


In [7]:
# 1: lat, 2: long
import geocoder
g = geocoder.ip('me')
user_address = g.latlng
print(user_address)

[37.566, 126.9784]


In [8]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    #lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

def test(cols):
    val = haversine(cols[0], cols[1], user_address[1], user_address[0])
    return val 

coordinates['distance'] = coordinates[['longitude','latitude']].apply(test, axis=1)

nearest_pharmacies = coordinates.sort_values(by='distance', ascending=True)[:15]

In [9]:
nearest_pharmacies.head(2)

,user_key,pharmercy_name,street_addr1,street_addr2,sido,gugun,dong,dong_code,longitude,latitude,distance
4787,1000075841,명약국,서울 중구 남대문로9길 51 (을지로1가),5층,서울특별시,중구,명동,1114055000,126.979499,37.566813,8.660742
4789,1000005172,홍신약국,서울 중구 남대문로9길 40 (다동),센터플레이스빌딩 1층 108-1호,서울특별시,중구,명동,1114055000,126.980383,37.567151,14.512949


In [10]:
import folium

user_address = g.latlng
coordinates['distance'] = coordinates[['longitude','latitude']].apply(test, axis=1)
nearest_pharmacies = coordinates.sort_values(by='distance', ascending=True)[:15]

locations = nearest_pharmacies[['latitude', 'longitude']]
locationlist = locations.values.tolist()

map = folium.Map(location=user_address, zoom_start=16)
for point in tqdm(range(0, 15)):
    folium.Marker(locationlist[point], popup=coordinates['pharmercy_name'][point]).add_to(map)
folium.Marker(user_address, popup='me', icon=folium.Icon(color='green')).add_to(map)
map

100%|██████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<?, ?it/s]


# User Interface Part

In [11]:
def picture_cap(on):   
    import cv2
    cap = cv2.VideoCapture(0) # video capture source camera (Here webcam of laptop) 
    ret,frame = cap.read() # return a single frame in variable `frame`
    on = True
    while on:
        cv2.imshow('img1',frame) #display the captured image
        if cv2.waitKey(1) & 0xFF == ord('y'): #save on pressing 'y' 
            cv2.imwrite('c1.png',frame)
            cv2.destroyAllWindows()
            break
    cap.release()

In [14]:
picture_cap(True)

In [32]:
picture_cap(True)

img_label = model.predict_classes('c1.png')
print(df_p.loc[img_label, '약명'])
print('-'*20)
print('1. 효능효과')
print('2. 사용상 주의사항')
print('3. 용법용량')
print('4. 성분정보')
print('5. 저장방법')
print('6. 주변 약국')
print('7. 궁금하시다면?')
print('8. 또 다른 약 재인식?')
print('9. 프로그램 완료')
question_list = []
option = int(input('Please, enter a menu 1~9: '))

on_2 = True

while on_2:
    if option == 1:
        print('-'*20)
        print('Specific Purpose')
        pd.options.display.max_colwidth = 100000
        print(df_p.loc[img_label, '효능효과'], sep='\n')
        print('-'*20)
        option = int(input('Please, enter a menu 1~9: '))
        print('-'*20)
        
    if option == 2:
        print('-'*20)
        print('사용상 주의사항')
        pd.options.display.max_colwidth = 100000
        print(df_p.loc[img_label, '사용상 주의사항'], sep='\n')
        print('-'*20)
        option = int(input('Please, enter a menu 1~9: '))
        print('-'*20)
        
    if option == 3:
        print('-'*20)
        print('Dose')
        pd.options.display.max_colwidth = 100000
        print(df_p.loc[img_label, '용법용량'], sep='\n')
        print('-'*20)
        print('8. Exit')
        option = int(input('Please, enter a menu 1~9: '))
        
    if option == 4:
        print('-'*20)
        print('성분정보')
        pd.options.display.max_colwidth = 100000
        print(df_p.loc[img_label, '성분정보'], sep='\n')
        print('-'*20)
        print('8. Exit')
        option = int(input('Please, enter a menu 1~9: '))
        
    if option == 5:
        print('-'*20)
        print('저장방법')
        pd.options.display.max_colwidth = 100000
        print(df_p.loc[img_label, '저장방법'], sep='\n')
        print('-'*20)
        option = int(input('Please, enter a menu 1~9: '))
        
    if option == 6:
        print('주변 약국 알려 드립니다!!')
        pharmacy_list = nearest_pharmacies[['pharmercy_name','distance']]
        print(pharmacy_list)
        print('-'*20)
        option = int(input('Please, enter a menu 1~9: '))
    
    if option == 7:
        print('서비스 향상을 위해 질문사항 남겨 최대한 답변 드리겠습니다.')
        question = str(input('질문 입력: '))
        question_list.append(question)
        print('-'*20)
        option = int(input('Please, enter a menu 1~9: '))
        
    if option == 8:
        picture_cap(True)
        img_label = model.predict_classes('c1.png')
        
    if option == 9:
        break
        
#import pickle
#with open("test.txt", "wb") as fp:
#    pickle.dump(question_list, fp)
#with open("test.txt", "rb") as fp:
 #   b = pickle.load(fp)

그린노즈 에스 
--------------------
1. 효능효과
2. 사용상 주의사항
3. 용법용량
4. 성분정보
5. 저장방법
6. 주변 약국
7. 궁금하시다면?
8. 또 다른 약 재인식?
9. 프로그램 완료


Please, enter a menu 1~9:  9
